# Создание RAG-приложения (Retrieval Augmented Generation)

Большие языковые модели (LLM) позволяют разрабатывать развитые вопросно-ответные приложения (или *Q&A приложения*), которые могут использовать для ответов заданные данные.
В основе таких приложений лежит методика известная как «генерация, дополненная извлечением данных» (retrieval-augmented generation или RAG).

В этом разделе приведен пример создания простого Q&A приложения на основе текстового источника данных.
Здесь вы также найдете пример типичной Q&A-архитектуры и сможете ознакомиться с дополнительными материалами, которые рассматривают более продвинутые методики, применяемые при разработке вопросно-ответных приложений.

## Что такое RAG?

RAG — это методика, которая позволяет расширить знания LLM дополнительными данными.

Большие языковые модели имеют представление о различных темах, но их знания ограничены общими данными, доступными на момент обучения.
Если вам нужно, чтобы модель знала о какой-то специфической информации или информации, которая появилась после ее обучения, вам понадобится предоставить ей соответствующие данные.
Предоставление таких данных и применение их в промпте это и есть RAG — генерация, дополненная извлечением данных.

GigaChain предоставляет компоненты для разработки вопросно-ответных приложений и поддержки RAG-методики в целом.

:::note

В этом разделе приводится пример разработки Q&A приложения, которое использует неструктурированные данные.
Пример использования RAG со структурированными данными — в разделе [Разработка Q&A приложения на основе SQL-данных](/docs/tutorials/sql_qa).

:::

## Архитектура RAG

В общем случае RAG приложение включает два компонента:

* Конвейер для загрузки данных из источника и их индексирование. Как правило, индексирование работает в автономном режиме.
* Цепочка RAG, которая в реальном времени обрабатывает запрос пользователя, извлекает соответствующие данные из индекса и передает их модели.

Процесс генерации ответа на основе необработанных данных можно представить следующим образом:

```mermaid
flowchart LR
    A(Индексирование)
    B(Извлечение данных и генерация)
    A -->  B
```

Индексирование включает этапы:

1. Загрузка документов с помощью [DocumentLoaders](/docs/concepts/#document-loaders).
2. Использование [разделителей текста](/docs/concepts/#text-splitters) для разбивки `Documents` на мелкие фрагменты. Это удобно как для индексации данных, так и для передачи их в модель, поскольку большие фрагменты труднее искать и использовать в рамках контекста модели.
3. Сохранение данных с помощью [векторных хранилищ](/docs/concepts/#vectorstores) и создание [эмбеддингов](/docs/concepts/#embedding-models) для поиска.

![index_diagram](../../../../static/img/rag_indexing.png)

Извлечение данных и генерация включает этапы:

1. Извлечение данных, которые соответствуют запросу пользователя, с помощью [ретривера](/docs/concepts/#retrievers).
2. Передача промпта, включающего вопрос пользователя и извлеченные данные, в [чат-модель](/docs/concepts/#chat-models) / [LLM](/docs/concepts/#llms) для генерации ответа.

![retrieval_diagram](../../../../static/img/rag_retrieval_generation.png)


## Подготовка

### Jupyter

Как и многие другие руководства в документации GigaChain это руководство основано на [блокноте Jupyter](https://jupyter.org/).
Блокноты предоставляют интерактивность, которая значительно упрощает изучение работы с LLM.

Об установке Jupyter читайте в [официальной документации](https://jupyter.org/install).

### Установка зависимостей

В примере используются модели GigaChat (чат-модель и эмбеддинги), а также векторное хранилище Chroma, но в своем приложении вы можете использовать любой из доступных компонентов.

Для работы с примером нужно установить пакеты:

In [ ]:
pip install --upgrade --quiet  gigachain-community gigachain-chroma bs4

Подробнее — в разделе [Установка](/ru/gigachain/get-started/installation).

<!--
### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls.
As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent.
The best way to do this is with [LangSmith](https://smith.langchain.com).

After you sign up at the link above, make sure to set your environment variables to start logging traces:

```shell
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."
```

Or, if in a notebook, you can set them with:

```python
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
```
-->

## Обзор решения

Пример демонстрирует вопросно-ответное приложение, которое генерирует ответы на основе [первой страницы документации GigaChain](https://developers.sber.ru/docs/ru/gigachain/overview).

Итоговое решение, которое включает конвейер индексации и цепочку RAG, может занимать всего около 20 строк кода:

In [96]:
import bs4
from langchain.prompts import load_prompt
from langchain_chroma import Chroma
from langchain_community.chat_models.gigachat import GigaChat
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings.gigachat import GigaChatEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

llm = GigaChat(
    credentials="<авторизационные_данные>",
    scope="GIGACHAT_API_PERS",
    model="GigaChat-Pro",
    verify_ssl_certs=False,
)

# Загрузка, разделение на части и индексация содержимого страницы.
loader = WebBaseLoader(
    web_paths=("https://developers.sber.ru/docs/ru/gigachain/overview",),
    bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("theme-doc-markdown markdown"))),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=GigaChatEmbeddings(
        credentials="<авторизационные_данные>",
        scope="GIGACHAT_API_PERS",
        verify_ssl_certs=False,
    ),
)

# Извлечение данных и генерация с помощью релевантных фрагментов данных.
retriever = vectorstore.as_retriever()

prompt = load_prompt("lc://prompts/retrievers/rag.yaml")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("Что такое GigaChain?")

'GigaChain — это библиотека Python, предназначенная для упрощения и автоматизации работы с нейросетевой моделью GigaChat и другими большими языковыми моделями (LLM). Она является адаптированной версией библиотеки LangChain, специально разработанной для работы с русским языком.'

In [97]:
# Очистка хранилища
vectorstore.delete_collection()

## Разбор решения

Ниже описаны этапы разработки приведенного примера.

## 1. Индексирование. Загрузка данных {#indexing-load}

Cначала загружается содержимое страницы.
Для этого используются загрузчики [DocumentLoaders](/docs/concepts#document-loaders), которые загружают данные из источника и возвращают список документов [`Documents`](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html).
Экземпляр `Document` — объект с полями:

* `page_content` (str) — содержимое страницы;
* `metadata` (dict) — метаданные.

Код в примере загружает HTML по заданному адресу и преобразовывает его в текст с помощью компонента [WebBaseLoader](/docs/integrations/document_loaders/web_base), который использует библиотеки `urllib` и `BeautifulSoup`.
Правила преобразования можно задать с помощью параметров парсера `BeautifulSoup` переданных в `bs_kwargs`.
Подробнее об изменении параметров — в [документации BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/#beautifulsoup).
Для представленного примера актуальны только HTML-теги с классом `theme-doc-markdown markdown`.

In [99]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Сохраняются только содержимое страницы документации.
bs4_strainer = bs4.SoupStrainer(class_="theme-doc-markdown markdown")
loader = WebBaseLoader(
    web_paths=("https://developers.sber.ru/docs/ru/gigachain/overview",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

len(docs[0].page_content)

3764

In [100]:
print(docs[0].page_content[:500])

GigaChain (GigaChat SDK)Обновлено 24 мая 2024GigaChain — это библиотека Python, которая позволяет упростить и автоматизировать работу с нейросетевой моделью GigaChat и другими большими языковыми моделями (LLM). GigaChain является версией библиотеки LangChain, которая адаптирована для работы с русским языком. Библиотека GigaChain обратно совместима с LangChain, что позволяет использовать ее не только для работы с GigaChat, но и при работе с другими LLM в различных комбинациях.Подробная документац


### Дополнительная информация

`DocumentLoader` — объект, который загружает данные в виде списка документов `Documents`.

* [Справка API](https://api.python.langchain.com/en/latest/document_loaders/langchain_core.document_loaders.base.BaseLoader.html) базового интерфейса.

<!--
* [Руководства по работе с `DocumentLoaders`](/docs/how_to#document-loaders).
[Integrations](../../../docs/integrations/document_loaders/): 160+
-->

## 2. Индексирование. Разделение данных {#indexing-split}

Размер загруженного документа превышает три тысячи символов. Текст такой длины может не умещаться в контекст модели.
Даже если модель уместит в контексте всю страницу, у нее могут быть проблемы с поиском информации в большом объеме входных данных.

Чтобы создать эмбеддинг из данных `Document` и сохранить его в векторном хранилище, документы нужно разделить на фрагменты.
Это позволит упростить извлечение релевантных частей страницы на этапе выполнения.

В примере документы делятся на фрагменты по 1000 символов с перекрытием в 200 символов между каждым фрагментом.
Перекрытие помогает избежать разделения утверждения и важного контекста, связанного с ним.
Для разделения текста используется [`RecursiveCharacterTextSplitter`](/docs/how_to/recursive_text_splitter), который рекурсивно делит документ с помощью общепринятых разделителей, таких как переводы строк `\n`, до тех пор, пока каждый фрагмент не достигнет нужного размера.
Используйте `RecursiveCharacterTextSplitter` для решения типичных задач, связанных с обработкой текста.

Атрибут `add_start_index=True` используется для сохранения индекса символа, с которого начинается каждый фрагмент начального документа `Document`.

In [101]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

5

In [102]:
len(all_splits[0].page_content)

997

In [103]:
all_splits[3].metadata

{'source': 'https://developers.sber.ru/docs/ru/gigachain/overview',
 'start_index': 2404}

### Дополнительная информация

`TextSplitter` — объект, который разделяет список документов `Document` на более мелкие фрагменты. Подкласс `DocumentTransformer`.

* Ознакомьтесь с [«контекстно-зависимыми разделителями»](/docs/how_to#text-splitters), которые сохраняют положение (контекст) каждого деления в исходном документе `Document`:
  * [Код (python или js)](/docs/integrations/document_loaders/source_code)
  * [Научные публикации](/docs/integrations/document_loaders/grobid)
* [Справка API](https://api.python.langchain.com/en/latest/base/langchain_text_splitters.base.TextSplitter.html) базового интерфейса.

`DocumentTransformer` — объект, который выполняет преобразование списка документов `Document`.
* [Руководства по работе с `DocumentTransformers`](/docs/how_to#text-splitters).
* [Справка API](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.transformers.BaseDocumentTransformer.html) базового интерфейса.

<!--
- [Integrations](../../../docs/integrations/document_transformers/)
-->

## 3. Индексирование. Хранение данных {#indexing-store}

Теперь полученные фрагменты текста нужно проиндексировать, чтобы иметь возможность искать данные в процессе выполнения.
Распространенным решением такой задачи является создание эмбеддингов для фрагментов и сохранение их в векторной базе данных (*векторном хранилище*).
Поиск по полученным фрагментам выполняется на основе сходства эмбеддинга запроса и эмбеддингов в хранилище.
Самая простая мера сходства — это косинусное сходство, при котором измеряется косинус угла между каждой парой эмбеддингов, представляющих собой высокоразмерные вектора.

Для создания эмбеддингов с помощью `GigaChatEmbeddings` и сохранения их в векторном хранилище Chroma достаточно одной команды.

In [104]:
from langchain_community.embeddings.gigachat import GigaChatEmbeddings
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=GigaChatEmbeddings(
        credentials="<авторизационные_данные>",
        scope="GIGACHAT_API_PERS",
        verify_ssl_certs=False,
    ),
)

### Дополнительная информация

`Embeddings` — обертка для работы с моделью, которая преобразует текст в эмбеддинги.

* [Руководство по работе с эмбеддингами](/docs/how_to/embed_text).
* [Справка API](https://api.python.langchain.com/en/latest/embeddings/langchain_core.embeddings.Embeddings.html) базового интерфейса.

<!--
- [Integrations](../../../docs/integrations/text_embedding/): 30+ integrations to choose from.
-->

`VectorStore` — обертка для работы с векторной базой данных, которая используется для хранения эмбеддингов и выполнения запросов.

* [Руководства по работе с векторными хранилищами](/docs/how_to/vectorstores).
* [Справка API](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html) для базового интерфейса.

<!--
- [Integrations](../../../docs/integrations/vectorstores/): 40+ integrations to choose from.
-->

На этом завершается индексирование.
К этому моменту в примере реализовано векторное хранилище, которое содержит фрагменты поста и может возвращать их в соответствии с запросом пользователя.

## 4. Извлечение данных и генерация. Извлечение данных{#retrieval-and-generation-retrieve}

Теперь добавьте логику приложения.
Приложение должно уметь:

1. Принимать вопрос пользователя;
2. Искать документы, соответствующие этому вопросу;
3. Передавать в модель вопрос и полученные документы;
4. Возвращать ответ.

Сначала определите логику поиска по документам.
Для этого GigaChain предоставляет интерфейс [Retriever](/docs/concepts#retrievers/) — обертку для работы с индексом.
Ретривер может возвращать подходящие документы `Documents` в ответ на строковый запрос.

Наиболее распространенным типом `Retriever` является [VectorStoreRetriever](/docs/how_to/vectorstore_retriever) — объект, который использует возможности поиска по сходству в векторном хранилище для облегчения получения данных.
Любой экземпляр `VectorStore` можно преобразовать в `Retriever` с помощью метода `VectorStore.as_retriever()`:

In [106]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs = retriever.invoke("Что такое GigaChain?")

len(retrieved_docs)

3

In [107]:
print(retrieved_docs[0].page_content)

GigaChain (GigaChat SDK)Обновлено 24 мая 2024GigaChain — это библиотека Python, которая позволяет упростить и автоматизировать работу с нейросетевой моделью GigaChat и другими большими языковыми моделями (LLM). GigaChain является версией библиотеки LangChain, которая адаптирована для работы с русским языком. Библиотека GigaChain обратно совместима с LangChain, что позволяет использовать ее не только для работы с GigaChat, но и при работе с другими LLM в различных комбинациях.Подробная документация для GigaChain доступна в репозитории.GigaChain находится на ранней стадии разработки. Будьте осторожны при использовании SDK в своих проектах, так как не все компоненты оригинальной библиотеки проверены на совместимость с GigaChat.Большая часть документации представлена на английском языке и находится в процессе локализации.Назначение﻿SDK упростит интеграцию вашего приложения с нейросетевой моделью GigaChat и поможет в таких задачах, как:Работа с промптами.Включая управление промптами и их


### Дополнительная информация

Как правило, для извлечения данных используются векторные хранилища, но существуют и другие подходы.

`Retriever` — объект, который возвращает документы `Document` на основе текстового запроса.

* [Дополнительная документация](/docs/how_to#retrievers) по интерфейсу и встроенным методикам получения данных. Которые, среди прочего, включают:
  * [`MultiQueryRetriever`](/docs/how_to/MultiQueryRetriever) генерирует вариации запроса, что позволяет повысить точность получаемых данных.
  * [`MultiVectorRetriever`](/docs/how_to/multi_vector) генерирует вариации эмбеддингов, для повышения точности получаемых данных.
  * [Self Query Retriever](/docs/how_to/self_queryy) фильтрует документы на основе метаданных при их получении из векторного хранилища.
* [Справка API](https://api.python.langchain.com/en/latest/retrievers/langchain_core.retrievers.BaseRetriever.html) для базового интерфейса.

<!--
    -   `Max marginal relevance` selects for [relevance and diversity](https://www.cs.cmu.edu/~jgc/publication/The_Use_MMR_Diversity_Based_LTMIR_1998.pdf) among the retrieved documents to avoid passing in duplicate context.
-->
<!--
-   [Integrations](../../../docs/integrations/retrievers/): Integrations
-->

## 5. Извлечение данных и генерация. Генерация {#retrieval-and-generation-generate}

Объедините полученный код в цепочку, которая:

* принимает вопрос;
* извлекает соответствующие документы;
* создает промпт;
* передает промпт в модель;
* анализирует выходные данные.

In [108]:
from langchain_community.chat_models.gigachat import GigaChat

llm = GigaChat(
    credentials="<авторизационные_данные>",
    scope="GIGACHAT_API_PERS",
    verify_ssl_certs=False,
    model="GigaChat-Pro",
)

Используйте промпт для RAG, который хранится хабе промптов [GigaChain](https://github.com/ai-forever/gigachain/tree/master/hub/prompts/retrievers/rag.yaml).

In [110]:
from langchain.prompts import load_prompt

prompt = load_prompt("lc://prompts/retrievers/rag.yaml")

example_messages = prompt.invoke(
    {"context": "образец контекста", "question": "образец вопроса"}
).to_messages()

example_messages

[HumanMessage(content='Твоя задача — отвечать на вопросы, чтобы помочь пользователю.\nДля ответов используй данные, извлеченные из источника.\nЕсли ты не знаешь ответа, просто скажи пользователю об этом.\nОтвечай коротко и ясно.\nТвой ответ должен быть не длиннее трех предложений.\n\nКонтекст: образец контекста\n\nВопрос: образец вопроса\n\nТвой ответ:')]

In [111]:
print(example_messages[0].content)

Твоя задача — отвечать на вопросы, чтобы помочь пользователю.
Для ответов используй данные, извлеченные из источника.
Если ты не знаешь ответа, просто скажи пользователю об этом.
Отвечай коротко и ясно.
Твой ответ должен быть не длиннее трех предложений.

Контекст: образец контекста

Вопрос: образец вопроса

Твой ответ:


Для создания цепочки используйте [Runnable-интерфейс LCEL](/docs/concepts#langchain-expression-language), который позволяет объединять компоненты и функции, получать потоковую, асинхронную и пакетную передачу данных.

In [112]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("Что такое GigaChain?"):
    print(chunk, end="", flush=True)

GigaChain - это библиотека Python, разработанная для упрощения и автоматизации работы с нейросетевой моделью GigaChat и другими большими языковыми моделями (LLM). Она является адаптированной версией библиотеки LangChain, предназначенной для работы с русским языком. GigaChain предоставляет стандартные интерфейсы и интеграции для работы с различными компонентами, а также позволяет создавать цепочки и агентов.

Разберем реализацию на LCEL, чтобы понять, что происходит.

Каждый из компонентов (`retriever`, `prompt`, `llm` и других) является экземпляром [Runnable](/docs/concepts#langchain-expression-language).
То есть они реализуют одни и те же методы, такие как `sync` и `async`, `.invoke`, `.stream` или `.batch`.
Это позволяет соединять их с помощью оператора конвейера `|` в [RunnableSequence](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableSequence.html) — последовательность, которая также является Runnable-объектом.

При обнаружении оператора `|` GigaChain автоматически приводит определенные объекты к Runnable.
Здесь `format_docs` приводится к [RunnableLambda](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableLambda.html), а словарь dict с `«context»` и `«question»` приводится к [RunnableParallel](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableParallel.html).
Самое важное на этом этапе то, что каждый объект является Runnable

На входе в `prompt` ожидается `dict` с ключами `«context»` и `«question»`.
Поэтому первый элемент этой цепочки создает Runnable-объекты, которые вычисляют оба ключа из входного вопроса:

* `retriever | format_docs` передает вопрос через ретривер, генерирующий объекты [Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html), а затем в `format_docs` для генерации строк;
* `RunnablePassthrough()` передает входной вопрос без изменений.

Таким образом, если собрать цепочку вида:

```python
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
)
```

То вызов метода `chain.invoke(question)` создаст отформатированный промпт, готовый к выводу.

:::tip

Тестирование с помощью таких подцепочек упрощает разработку на LCEL.

:::

Работа цепочки завершается обращение к `llm`, которое запускает вывод.
После чего вызывается `StrOutputParser()`, который извлекает строковое содержимое из выходного сообщения модели.

### Встроенные цепочки

При желании вы можете воспользоваться встроенными в GigaChain функциями, которые делают то же самое, что и описанная реализация на LCEL:

* [`create_stuff_documents_chain`](https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html) определяет, как извлеченный контекст будет подаваться в промпт и LLM. Цепочка помещает в промпт весь найденный контекст без суммаризации или другой обработки. По большому счету она реализует описанную цепочку `rag_chain`, с входными ключами `context` и `input`. Функция генерирует ответ, используя извлеченный контекст и запрос.
* [`create_retrieval_chain`](https://api.python.langchain.com/en/latest/chains/langchain.chains.retrieval.create_retrieval_chain.html) добавляет шаг извлечения и распространяет извлеченный контекст по цепочке, предоставляя его вместе с окончательным ответом. Цепочка имеет входной ключ `input` и включает `input`, `context` и `answer` в свой вывод.

In [113]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "Твоя задача — отвечать на вопросы, чтобы помочь пользователю. "
    "Для ответов используй данные, извлеченные из источника. "
    "Если ты не знаешь ответа, просто скажи пользователю об этом. "
    "Твой ответ должен быть не длиннее трех предложений."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "Что такое GigaChain?"})
print(response["answer"])

GigaChain — это библиотека Python, предназначенная для упрощения работы с нейросетевой моделью GigaChat и другими большими языковыми моделями. Она позволяет создавать цепочки и агентов, управлять промптами и работать с памятью.


#### Получение источников

Зачастую в вопросно-ответных приложениях важно показать пользователю источники, использованные для генерации ответа.
Встроенная цепочка `create_retrieval_chain` будет добавлять полученные источники в вывод, в ключе `«context»`:

In [114]:
for document in response["context"]:
    print(document)
    print()

page_content='GigaChain (GigaChat SDK)Обновлено 24 мая 2024GigaChain — это библиотека Python, которая позволяет упростить и автоматизировать работу с нейросетевой моделью GigaChat и другими большими языковыми моделями (LLM). GigaChain является версией библиотеки LangChain, которая адаптирована для работы с русским языком. Библиотека GigaChain обратно совместима с LangChain, что позволяет использовать ее не только для работы с GigaChat, но и при работе с другими LLM в различных комбинациях.Подробная документация для GigaChain доступна в репозитории.GigaChain находится на ранней стадии разработки. Будьте осторожны при использовании SDK в своих проектах, так как не все компоненты оригинальной библиотеки проверены на совместимость с GigaChat.Большая часть документации представлена на английском языке и находится в процессе локализации.Назначение﻿SDK упростит интеграцию вашего приложения с нейросетевой моделью GigaChat и поможет в таких задачах, как:Работа с промптами.Включая управление про

### Дополнительная информация

#### Выбор модели

`ChatModel` — чат-модель, в основе которой лежит LLM. Объект принимает на вход последовательность сообщений и возвращает ответ модели в виде сообщения.

* [Документация](/docs/how_to#chat-models);
* [Справка API](https://api.python.langchain.com/en/latest/language_models/langchain_core.language_models.chat_models.BaseChatModel.html) для базового интерфейса.

<!--
- [Integrations](../../../docs/integrations/chat/): 25+ integrations to choose from.
-->

`LLM` — LLM, которая принимает и возвращает строку.

* [Документация](/docs/how_to#llms);
* [Справка API](https://api.python.langchain.com/en/latest/language_models/langchain_core.language_models.llms.BaseLLM.html) для базового интерфейса.

<!--
- [Integrations](../../../docs/integrations/llms): 75+ integrations to choose from.
-->

Руководство по реализации [RAG с помощью локальных моделей](/docs/tutorials/local_rag).

#### Настройка промпта

В примере выше RAG-промпт загружается из [хаба промптов GigaChain](https://github.com/ai-forever/gigachain/blob/master/hub/prompts/retrievers/rag.yaml).
Если нужно вы можете использовать собственный промпт:

In [115]:
from langchain_core.prompts import PromptTemplate

template = """Твоя задача — отвечать на вопросы, чтобы помочь пользователю.
Для ответов используй данные, извлеченные из источника.
Если ты не знаешь ответа, просто скажи пользователю об этом.
Отвечай коротко и ясно.
Твой ответ должен быть не длиннее трех предложений.

Контекст: {context}

Вопрос: {question}

Твой ответ:"""

custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("Что такое GigaChain?")

'GigaChain - это библиотека Python, разработанная для упрощения и автоматизации работы с нейросетевой моделью GigaChat и другими большими языковыми моделями (LLM). Она является адаптированной версией библиотеки LangChain, предназначенной для работы с русским языком. GigaChain предоставляет стандартные интерфейсы и интеграции для работы с различными компонентами, а также инструменты для создания цепочек и агентов.'

## Смотрите также

* [Получение исходных документов](/docs/how_to/qa_sources).
* [Потоковая передача выходных данных и промежуточных шагов](/docs/how_to/streaming).
* [Добавление истории чата](/docs/how_to/message_history).